# Kitaev Model: Toric Code

Kitaev Hamiltonian:
$
\begin{equation}
H_K = - \sum_p B_p - \sum_s A_s
\end{equation}
$

With operators defined as:

$
\begin{align}
B_p &= \Pi_{(ij) \in p} \sigma_{(ij)}^z \\
A_s &= \Pi_{(ij) \in s} \sigma_{(ij)}^x \\
\end{align}
$

where $p$ is a plaquette of the lattice and $s$ is a star vertex of the lattice.

In [3]:
import numpy as np

Consider an $N=2$ torus, meaning there are $2N^2 = 8$ spin sites and periodic boundary conditions

In [30]:
Nspins = 8  # number of spins on lattice
Np = 4  # number of plaquettes
Ns = 4  # number of star vertices

Generate $B_p$ and $A_s$ operators. Since there are only $8$ operators in total, I'll just write them in by hand. Later, we can automate it based off a graph representing the spin sites.

In [34]:
x = np.array([[0,1],[1,0]])
z = np.array([[1,0],[0,-1]])
iden = np.identity(2)

Bp_s = []  # list of Bp operators indexed from 0 to 3
Bp_s.append(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(iden, z), z), z), iden), z), iden), iden))
Bp_s.append(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(z, iden), z), z), z), iden), iden), iden))
Bp_s.append(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(iden, z), iden), iden), iden), z), z), z))
Bp_s.append(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(z, iden), iden), iden), z), iden), z), z))

As_s = []  # list of Bp operators indexed from 0 to 3
As_s.append(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(iden, iden), x), iden), x), x), x), iden))
As_s.append(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(iden, iden), iden), x), x), x), iden), x))
As_s.append(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(x, x), x), iden), iden), iden), x), iden))
As_s.append(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(np.kron(x, x), iden), x), iden), iden), iden), x))

Check $B_p$ and $A_s$ operators commute according to

$
\begin{align}
[B_p, B_{p\prime}] &= 0 \\
[A_s, A_{s\prime}] &= 0 \\
[B_p, A_s] &= 0 \\
\end{align}
$

Check that periodic boundary condition

$
\begin{align}
\Pi_s A_s &= 1 \\
\Pi_p B_p &= 1 \\
\end{align}
$

holds.


In [58]:
# Check commutation relations. Print if there something does NOT commute

# Commutation between different Bp operators
for i in range(len(Bp_s) - 1):
    if not np.array_equal(np.matmul(Bp_s[i], Bp_s[i+1]), np.matmul(Bp_s[i+1], Bp_s[i])):
        print('Bp', i, 'does not commute with Bp', i+1)

# Commutation between different As operators
for i in range(len(As_s) - 1):
    if not np.array_equal(np.matmul(As_s[i], As_s[i+1]), np.matmul(As_s[i+1], As_s[i])):
        print('As', i, 'does not commute with As', i+1)

# Commutation between different Bp and As operators
for i in range(len(As_s)):
    for j in range(len(Bp_s)):
        if not np.array_equal(np.matmul(As_s[i], Bp_s[j]), np.matmul(Bp_s[j], As_s[i])):
            print('As', i, 'does not commute with As', i+1)

# Check periodic boundary condition
Bp_product = np.identity(Bp_s[0].shape[0])
for i in range(len(Bp_s)):
    Bp_product = np.matmul(Bp_product, Bp_s[i])
print('Is the product of Bp operators equal to the identity matrix?:', np.array_equal(Bp_product, np.identity(Bp_s[0].shape[0])))

As_product = np.identity(As_s[0].shape[0])
for i in range(len(As_s)):
    As_product = np.matmul(As_product, As_s[i])
print('Is the product of As operators equal to the identity matrix?:', np.array_equal(As_product, np.identity(As_s[0].shape[0])))

Is the product of Bp operators equal to the identity matrix?: True
Is the product of As operators equal to the identity matrix?: True


Looks like everything checks out. Now to compute the Hamiltonian $H_K$

In [222]:
# Sum all the Bp and As operators
H_k = -sum(Bp_s) - sum(As_s)
H_k

array([[-4., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.],
       ...,
       [-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -0.],
       [-0., -0., -0., ..., -0., -0., -4.]])

# Compute eigenvalues and eigenvectors of Hamiltonian

In [223]:
# Compute eigenvalues and eigenvectors
# Should be four ground states because lattice is on a torus
# 1st excited state energy must be 4 above ground state again due to torus manifold
eigen_vals, eigen_vecs = np.linalg.eig(H_k)  # np.real is used to removed 1e-17j values

# Clean up eigenvalues
eigen_vals = np.round(np.real(eigen_vals), 4)
unqiue_eigvals, counts = np.unique(eigen_vals, return_counts=True)
print(unqiue_eigvals, counts)

# normalize eigenvectors
eigen_vecs = np.round(np.real(eigen_vecs), 4)
norms = np.linalg.norm(eigen_vecs, ord=2, axis=1)
for i, norm in enumerate(norms):
    eigen_vecs[i] = eigen_vecs[i] / norm
print('average norm:', np.mean(np.linalg.norm(eigen_vecs, ord=2, axis=1)))

[-8. -4. -0.  4.  8.] [  4  48 152  48   4]
average norm: 1.0


In [224]:
# 4 ground states and check orthogonality
ground_states = []
ground_energy = eigen_vals.min()
for i, val in enumerate(eigen_vals):
    if val == ground_energy:
        ground_states.append(eigen_vecs[i])

# Inner product should be 1 between ground state 0 and itself and 0 between ground state 0 and the other 3 ground states  
for j in range(len(ground_states)):
    inner_product = np.matmul(np.transpose(ground_states[0]), ground_states[j])
    print(inner_product)


0.9999999999999999
-0.03990973939708617
-0.003005483570822708
-0.13505679902182943
